In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

In [3]:
import json
import tensorflow as tf
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
import warnings
from keras.preprocessing.text import tokenizer_from_json

warnings.filterwarnings("ignore")
warnings.simplefilter(action="ignore", category=FutureWarning)

from dataset import *
from lstm import *

dataset = load_tokenized_sentences("../datasets/words/bajki-raw.pickle")

with open("../datasets/words/books-bajki-raw-tokenizer_100000.json", "r") as f:
    data = json.load(f)
    tokenizer = tokenizer_from_json(data)

print(
    {
        "size": len(dataset),
        "tokenizer": tokenizer.num_words,
        "str": dataset[1][:500],
        "tokenizer": tokenizer.num_words,
    }
)

# Use post padding
# https://stackoverflow.com/questions/71545569/why-does-post-padding-train-faster-than-pre-padding

# Post vs Pre padding
# https://arxiv.org/pdf/1903.07288.pdf

# In the end - pre is better

{'size': 41, 'tokenizer': 100000, 'str': 'była raz mała słodka dzieweczka, którą kochał każdy, kto ją tylko ujrzał, a najwięcej kochała ją babcia  nie wiedziała wprost, co jej dać. pewnego razu podarowała jej kapturek z czerwonego aksamitu, a dziewczynce tak się ten kapturek podobał, że nie chciała nosić żadnego innego, toteż nazwano ją czerwonym kapturkiem.\npewnego razu rzekła matka do czerwonego kapturka:\n oto masz, dziecko, w koszyku placek i flaszkę wina, zanieś to babci, która jest chora i słaba, i ucieszy się bardzo tym podarunkie'}


In [4]:
class_weights = calculate_class_weights(tokenizer, alpha=0.4) 
SKIP = 2
total_words = tokenizer.num_words
batch_size = 256

padding: Literal["pre", "post"] = "pre"
dataset_size = calculate_dataset_size(dataset, tokenizer, LEN_MAX_LIMIT, SKIP)
gen = dataset_generator(
    dataset, tokenizer, LEN_MIN_LIMIT, LEN_MAX_LIMIT, SKIP, batch_size, padding=padding
)
a, b = next(gen)
print(a.shape, b.shape)
print(a[0], b[0])
print('class_weights', len(class_weights), class_weights[[1,2,3,4,5,-5,-4,-3,-2,-1]])

(256, 50) (256,)
[    0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     7 12597 38229    10   121 28720   356
   136 15801  2735    17  1709  9864     5 57538    10     8    31   144
    76     4 11959   338   726    31  5992   997     2     3  2041     3
   630     1] 6183
class_weights 100001 [ 1.          1.10900746  1.14756841  1.22807962  1.26507838 73.39486412
 73.39486412 73.39486412 73.39486412 73.39486412]


In [5]:
dataset_val = load_tokenized_sentences("../datasets/words/books-bajki-raw-val.pickle")

dataset_size_val = calculate_dataset_size(dataset_val, tokenizer, LEN_MAX_LIMIT, SKIP)
gen_val = dataset_generator(
    dataset_val,
    tokenizer,
    LEN_MIN_LIMIT,
    LEN_MAX_LIMIT,
    SKIP,
    batch_size,
    padding=padding,
)
a, b = next(gen_val)
print(a.shape, b.shape)
print(a[0], b[0])
print(tokenizer.sequences_to_texts([a[0]]), tokenizer.sequences_to_texts([[b[0]]]))

(256, 50) (256,)
[    0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0 12075   210 84220     2   401    95  6863    20
     5   868   130     1    20    35     4  4218    34   438     7    14
     5  1949] 210
['pytanie. - plącze się koło tych widzieliśmy go na własne oczy i go jeszcze nie wiemy, czy należy do ale na wszelki'] ['-']


In [6]:
print(total_words, LEN_MAX_LIMIT)
model = create_model_v2(LEN_MAX_LIMIT, total_words, class_weights)
model.summary()

100000 50


2023-09-10 23:40:57.824419: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-09-10 23:40:57.825168: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-09-10 23:40:57.825775: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

RuntimeError: pybind11::error_already_set: MISMATCH of original and normalized active exception types: ORIGINAL _NotOkStatusException REPLACED BY KeyboardInterrupt: <EMPTY MESSAGE>

At:
  /home/adrian/anaconda3/envs/tf_311/lib/python3.11/site-packages/tensorflow/python/eager/core.py(36): __init__
  /home/adrian/anaconda3/envs/tf_311/lib/python3.11/site-packages/tensorflow/python/ops/gen_math_ops.py(461): add_v2
  /home/adrian/anaconda3/envs/tf_311/lib/python3.11/site-packages/tensorflow/python/ops/math_ops.py(4022): add
  /home/adrian/anaconda3/envs/tf_311/lib/python3.11/site-packages/tensorflow/python/util/dispatch.py(1176): op_dispatch_handler
  /home/adrian/anaconda3/envs/tf_311/lib/python3.11/site-packages/tensorflow/python/util/traceback_utils.py(150): error_handler
  /home/adrian/anaconda3/envs/tf_311/lib/python3.11/site-packages/tensorflow/python/ops/stateless_random_ops.py(523): stateless_random_uniform
  /home/adrian/anaconda3/envs/tf_311/lib/python3.11/site-packages/tensorflo

In [ ]:
model.fit(
    gen,
    steps_per_epoch=dataset_size // batch_size,
    epochs=25,
    verbose=1,
    validation_data=gen_val,
    validation_steps=128,
    callbacks=[
        tf.keras.callbacks.ModelCheckpoint(
            "../lstm_models/model_best_5.h5",
            monitor="loss",
            save_best_only=True,
            save_weights_only=False,
        ),
        # ReduceLROnPlateau(factor=0.5, patience=2, monitor="val_loss"),
    ]
    + [
        PredictCallback(
            "dawno temu czerwony kapturek poszedł do lasu",
            60,
            LEN_MAX_LIMIT,
            tokenizer,
            padding,
            _t,
        )
        for _t in [0, 0.1, 0.2, 0.3, 0.5, 0.8, 1.0]
    ],
)


"""
Była raz mała słodka dzieweczka, którą kochał każdy, kto ją tylko ujrzał, a najwięcej kochała ją babcia — nie wiedziała wprost, co jej dać. Pewnego razu podarowała jej kapturek z czerwonego aksamitu, a dziewczynce tak się ten kapturek podobał, że nie chciała nosić żadnego innego, toteż nazwano ją Czerwonym Kapturkiem.
Pewnego razu rzekła matka do Czerwonego Kapturka:
— Oto masz, dziecko, w koszyku placek i flaszkę wina, zanieś to babci, która jest chora i słaba, i ucieszy się bardzo tym podarunkiem. Idź zaraz, póki nie ma wielkiego upału, a idąc nie biegaj i nie zbaczaj z drogi, bo mogłabyś upaść i stłuc butelkę. Kiedy zaś wejdziesz do pokoju, nie zapomnij powiedzieć babci „dzień dobry” i nie rozglądaj się wpierw po wszystkich kątach.
— Zrobię wszystko, jak każesz — przyrzekł Czerwony Kapturek mamusi.
Babcia mieszkała w lesie, o pół godzinki od wsi. Kiedy dziewczynka weszła do lasu, spotkała wilka. Ale Czerwony Kapturek nie wiedział, że to takie złe zwierzę, i nie bał się go.
— Dzień dobry, Czerwony Kapturku — rzekł wilk.
"""

Start epoch 0 of training; Temperature: 0.0 Generated text: dawno temu czerwony kapturek poszedł do lasu niezwykłej niezwykłej niezwykłej niezwykłej niezwykłej niezwykłej znaki? znaki? znaki? znaki? śpiącego, śpiącego, śpiącego, śpiącego, śpiącego, śpiącego, śpiącego, śpiącego, śpiącego, śpiącego, śpiącego, śpiącego, śpiącego, śpiącego, śpiącego, śpiącego, śpiącego, śpiącego, uszły uszły uszły uszły uszły uszły uszły synowie, synowie, synowie, synowie, chocim synowie, posuwający uniżenie uniżenie uniżenie uniżenie knoty tysiącznych tysiącznych tysiącznych tysiącznych tysiącznych tysiącznych porównywać porównywać porównywać porównywać koszule, koszule, krawca.
Start epoch 0 of training; Temperature: 0.1 Generated text: dawno temu czerwony kapturek poszedł do lasu głazie. skrzypienie stajnia najznakomitszego językach czarnota konwulsyjnym wicher. odłóż podsłuchiwać starsi śni przestwór nastały zgiął zebrał, cichu.

 pracą. zaspokaja sypiałem sztyletu pustym, silnym, linety, biją co! mefr

'\nByła raz mała słodka dzieweczka, którą kochał każdy, kto ją tylko ujrzał, a najwięcej kochała ją babcia — nie wiedziała wprost, co jej dać. Pewnego razu podarowała jej kapturek z czerwonego aksamitu, a dziewczynce tak się ten kapturek podobał, że nie chciała nosić żadnego innego, toteż nazwano ją Czerwonym Kapturkiem.\nPewnego razu rzekła matka do Czerwonego Kapturka:\n— Oto masz, dziecko, w koszyku placek i flaszkę wina, zanieś to babci, która jest chora i słaba, i ucieszy się bardzo tym podarunkiem. Idź zaraz, póki nie ma wielkiego upału, a idąc nie biegaj i nie zbaczaj z drogi, bo mogłabyś upaść i stłuc butelkę. Kiedy zaś wejdziesz do pokoju, nie zapomnij powiedzieć babci „dzień dobry” i nie rozglądaj się wpierw po wszystkich kątach.\n— Zrobię wszystko, jak każesz — przyrzekł Czerwony Kapturek mamusi.\nBabcia mieszkała w lesie, o pół godzinki od wsi. Kiedy dziewczynka weszła do lasu, spotkała wilka. Ale Czerwony Kapturek nie wiedział, że to takie złe zwierzę, i nie bał się go.\n—

In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

from tqdm import tqdm
from keras.models import Model, load_model
from lstm import *
from keras import losses
from keras.preprocessing.text import tokenizer_from_json
import json 

with open("../datasets/words/books-bajki-raw-tokenizer_100000.json", "r") as f:
    data = json.load(f)
    tokenizer = tokenizer_from_json(data)


model: Model = load_model("../lstm_models/model_best_5.h5", custom_objects={"WeightedSCCE":  losses.SparseCategoricalCrossentropy})
text_seed = "dawno temu czerwony kapturek poszedł do lasu"

for temperature in tqdm([0, 0.1, 0.2, 0.3, 0.5, 0.8, 1.0]):
    pr = PredictCallback(text_seed, 250, LEN_MAX_LIMIT, tokenizer,'pre',  temperature, model)
    text = pr.generate_text()
    with open(f"../generated_texts/lstm/text_temperature_{temperature:.2f}", "w") as f:
        f.write(f"Seed: {text_seed}\n")
        f.write(text)

100%|██████████| 7/7 [00:21<00:00,  3.12s/it]
